# データベースに侵入しようとするハッカーを撃退しよう


## 注意!

- 実環境において攻撃を仕掛けることは不正アクセス禁止法により法律で固く禁じられています。
このコードは教育目的で作成しており、仮想的な環境で実行しています。(このコードは攻撃用ではなく防御用です。)

---

### ・セクション1: データベースの作成関数とログ保存関数を準備しよう。

In [1]:
#必要ライブラリのインポート
import random
import time
import json
from datetime import datetime, timezone
from getpass import getpass

In [2]:
#データベース作成関数
def create_db(num_users):
    db = {}
    for i in range(1, num_users+1):
        username = f"user{i}"
        #3桁のランダムな整数パスワード
        password = f"{random.randint(0, 999):03d}"
        db[username] = {
            "password": password,
            "email": f"{username}@example.com",
            "notes": f"Sample note for {username}"
        }
    return db

In [3]:
#ログ機能の作成

#グローバル変数の定義
login_attempts_log = []   #ログイン情報を保管しておく配列を作る
lock_state = {}           #状態の辞書
LOCK_THRESHOLD = 3        # 3回でロック
LOCK_DURATION_SEC = 60    # ロック期間：60秒

#ログ情報の作成関数
def log_attempt(username, success, source="local", note=""):
    #辞書entryを作成して、情報を保存
    entry = {
        "time": datetime.now(timezone.utc).isoformat(),
        "username": username,
        "success": bool(success),
        "source": source,
        "note": note
    }
    #辞書を保管する配列に追加
    login_attempts_log.append(entry)

#ログインに失敗したことを記録
def register_failed_attempt(username):
    #ロック情報を保管する辞書の作成
    s = lock_state.get(username, {"failed":0, "locked_until":None})
    s["failed"] += 1
    if s["failed"] >= LOCK_THRESHOLD:
        s["locked_until"] = time.time() + LOCK_DURATION_SEC
    lock_state[username] = s

#ロック状態の作成関数
def is_locked(username):
    #userキーに対応する辞書をstateとして変数に保存
    state = lock_state.get(username)

    #キーに対応するstateが存在しない場合
    if not state:
        return False
    
    #stateのlocked_untilキーの値を取得 
    locked_until = state.get("locked_until")

    #ロック期間中(ロック状態がtrue)
    if locked_until and time.time() < locked_until:
        return True
        
    # ロック期間経過したらリセット
    if locked_until and time.time() >= locked_until:
        lock_state.pop(username, None)
        return False

    return False

#状態のリセット
def reset_attempts(username):
    lock_state.pop(username, None)

[補足]
ロック情報の保存形式の例(辞書の中にさらに辞書を作成することが可能)
{"userA": {"failed": 3, "locked_until": None}, "userB": {"failed": 5, "locked_until": 1698213000.0},・・・}


point) 上の関数たちの中身は理解しないで大丈夫です。それぞれが何をする関数なのかだけ理解しておきましょう。
```
- log_attempt(#引数は省略)・・・ログイン情報の記録(ログ)を作成して保存。
  
- register_failed_attempt(username)・・・ログインに失敗したことを記録してロック状態をon

- is_locked(username)・・・ロック状態がon(true)かoff(false)かを判断

- reset_attempts(username)・・・ロックの情報を削除
```

---

### ・セクション2: ログインを行う関数を作成しよう。

今回のシナリオ:

- 攻撃者・・・「データベースに侵入してデータをRSAで暗号化してやる!」 (ランサムウェアを仕掛けようとしている)

- 防御側・・・「パスワード突破できるもんならやってみろ！」

- 攻撃者・・・「繰り返しを行うプログラムで000から順に試して行ってやるよ！」(ブルートフォース攻撃!)

- 防御側・・・「そんなことさせるか！ログを監視して侵入を防いでやる!」

In [13]:
#安全なログイン関数
def safe_login(db, max_inputs=5, source="local"):
    #while(ずっと繰り返し)ではなく、for文を使用してブルートフォース攻撃を防御しよう！！
    for attempt in range(max_inputs):
        username = input("ユーザー名: ").strip()
            
        #ユーザー名が存在しない場合
        if  username not in db:
            #管理者権限 (これが開発者が削除し忘れたバックドア: if ~ else文を削除)
            if username == "user0":
                return db
            else:
                print("ユーザーが存在しません")
                log_attempt(username, False, source, note="no such user")
                continue
          
        #アカウントがロック時の動作
        if is_locked(username):
            print("アカウントは一時ロックされています。しばらく待ってから再試行してください。")
            log_attempt(username, False, source, note="locked")
            return False

        #パスワードの入力(パスワードが見えないようになる)
        password = getpass("パスワード（3桁）: ").strip()

        #ログイン成功時の処理
        if password == db[username]["password"]:
            print("ログイン成功")
            log_attempt(username, True, source)
            reset_attempts(username)
            return True
        
        #ログイン失敗時の例
        else:
            print("パスワードが違います")
            log_attempt(username, False, source, note="worng password")
            #以下2行を空白にして問題にする
            register_failed_attempt(username)
            s = lock_state.get(username)
            if s and s.get("locked_until"):
                print(f"アカウントがロックされました（{LOCK_DURATION_SEC}秒）")
                return False
                
    print("試行回数を超えました")
    return False

In [9]:
#データベースの作成
db = create_db(3)
print("== DataBase sample ==")
print(json.dumps(db, indent=2, ensure_ascii=False))

== DataBase sample ==
{
  "user1": {
    "password": "222",
    "email": "user1@example.com",
    "notes": "Sample note for user1"
  },
  "user2": {
    "password": "518",
    "email": "user2@example.com",
    "notes": "Sample note for user2"
  },
  "user3": {
    "password": "425",
    "email": "user3@example.com",
    "notes": "Sample note for user3"
  }
}


In [10]:
#ログ確認用のユーティリティ
def show_logs(limit=20):
    print("== Login Attempts Log (recent) ==")
    for e in login_attempts_log[-limit:]:
        print(json.dumps(e, ensure_ascii=False))

def show_lock_state():
    print("== Lock State ==")
    for k, v in lock_state.items():
        print(k, v)

In [12]:
#正しいパスワードでログインしてみよう
print("=== 正しいパスワードを試す ===")
print("ユーザーの一覧:", list(db.keys()))
safe_login(db)

=== 正しいパスワードを試す ===
ユーザーの一覧: ['user1', 'user2', 'user3']


ユーザー名:  user1
パスワード（3桁）:  ········


ログイン成功


True

In [16]:
#間違ったパスワードでログインしてみよう
print("=== 間違ったパスワードを試す ===")
print("ユーザーの一覧:", list(db.keys()))
safe_login(db)

=== 間違ったパスワードを試す ===
ユーザー一覧: ['user1', 'user2', 'user3']


ユーザー名:  user1
パスワード（3桁）:  ········


パスワードが違います


ユーザー名:  111


ユーザーが存在しません


...

ユーザー名:  


ユーザーが存在しません


ユーザー名:  user1
パスワード（3桁）:  ········


パスワードが違います
試行回数を超えました


False

In [17]:
#ログを確認しよう
show_logs()
print("===================================\n")
show_lock_state()

== Login Attempts Log (recent) ==
{"time": "2025-10-22T10:27:53.042830+00:00", "username": "795", "success": false, "source": "local", "note": "no such user"}
{"time": "2025-10-22T10:28:00.305590+00:00", "username": "user1", "success": true, "source": "local", "note": ""}
{"time": "2025-10-22T10:28:12.367984+00:00", "username": "user1", "success": false, "source": "local", "note": "worng password"}
{"time": "2025-10-22T10:28:13.843910+00:00", "username": "111", "success": false, "source": "local", "note": "no such user"}
{"time": "2025-10-22T10:28:14.792665+00:00", "username": "111", "success": false, "source": "local", "note": "no such user"}
{"time": "2025-10-22T10:28:16.322989+00:00", "username": "", "success": false, "source": "local", "note": "no such user"}
{"time": "2025-10-22T10:28:20.020075+00:00", "username": "user1", "success": false, "source": "local", "note": "worng password"}

== Lock State ==
user1 {'failed': 2, 'locked_until': None}


---

### 演習

#### ・レッドチーム(ハッキング側)・・・上のログイン関数には脆弱性が残っています。脆弱性を見つけて攻撃を仕掛けてみましょう。

#### ・ブルーチーム(防御側)・・・脆弱性の修正を行い、ハッカーの攻撃を防ぎましょう

- ヒント
  ここにはバックドアと呼ばれるセキュリティの穴があります。管理者権限に注目しましょう.
  ここから侵入するとログにも残らないのでこのままにしておくと非常に危険です.

In [ ]:
#レッドチームは侵入を試す。ブルーチームは上のsafe_login関数を修正後に実行して試す。
print("=== 侵入が可能か試す ===")
print("ユーザーの一覧:", list(db.keys()))
safe_login(db)